**NLP stands for Natural Language Processing which is the task of mining the text and find out some meaningful insights like Sentiments, Named Entity, Topic of Discussion and even Summary of the text.**

With this IMDB dataset we will do the Sentiment Analysis.

Firstly,we will apply some text cleaning techniques i.e do some text pre-processing since textual data is in free form.

Since we cannot apply text to our Machine Learning Model directly we have to convert the text into mathematical form (vector representation) and explore different Vectorization / Text Encoding Techniques. 

***Importing basic libraries***

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

***Loading dataset***

In [4]:
dataset = pd.read_csv('../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

print(dataset.shape)
dataset.head(10)


(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [5]:
dataset.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


**There are two columns - review and sentiment.
Sentiment is the target column that we have to predict further.**

In [7]:
dataset['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

*Dataset is balanced and has equal number of positive and negative sentiments.*

**Taking one review as sample and understanding the need of cleaning the text.**

In [8]:
review = dataset['review'].iloc[1]
review

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

**In general any NLP task involves the following text cleaning techniques -**
1. Removal of HTML contents like "\<br>"
2. Removal of punctuation and special characters.
3. Removal of stopwords like the, when, how etc which do not offer much insights.
4. Stemming/Lemmatization techniques to have the stem word of the words having multiple forms of words.
5. Vectorization - encoding the textual data to numerical form after cleaning.
6. Fitting the data to ML model.




**Applying the techniques to sample data to understand the process first -**

1. Removal of HTML contents

In [9]:
!pip install bs4


     |████████████████████████████████| 115 kB 1.3 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=7143a09c22b9c6a25583ac3b21ffe7aabf604a1a59b16c41f04cd1f773371a4c
  Stored in directory: /root/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [10]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(review, 'html.parser')
review = soup.get_text()
review

'A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well done.'

**HTML tags are removed now remove everything except the lowercase/uppercase letters using regular expressions**

In [11]:
import re
review = re.sub('\[[^]]*/]',' ',review)
review = re.sub('[^a-zA-Z]',' ',review)
review

'A wonderful little production  The filming technique is very unassuming  very old time BBC fashion and gives a comforting  and sometimes discomforting  sense of realism to the entire piece  The actors are extremely well chosen  Michael Sheen not only  has got all the polari  but he has all the voices down pat too  You can truly see the seamless editing guided by the references to Williams  diary entries  not only is it well worth the watching but it is a terrificly written and performed piece  A masterful production about one of the great master s of comedy and his life  The realism really comes home with the little things  the fantasy of the guard which  rather than use the traditional  dream  techniques remains solid then disappears  It plays on our knowledge and our senses  particularly with the scenes concerning Orton and Halliwell and the sets  particularly of their flat with Halliwell s murals decorating every surface  are terribly well done '

**Now converting everthing into lowercase**

In [12]:
review =  review.lower()
review

'a wonderful little production  the filming technique is very unassuming  very old time bbc fashion and gives a comforting  and sometimes discomforting  sense of realism to the entire piece  the actors are extremely well chosen  michael sheen not only  has got all the polari  but he has all the voices down pat too  you can truly see the seamless editing guided by the references to williams  diary entries  not only is it well worth the watching but it is a terrificly written and performed piece  a masterful production about one of the great master s of comedy and his life  the realism really comes home with the little things  the fantasy of the guard which  rather than use the traditional  dream  techniques remains solid then disappears  it plays on our knowledge and our senses  particularly with the scenes concerning orton and halliwell and the sets  particularly of their flat with halliwell s murals decorating every surface  are terribly well done '

**Now removal of Stopwords (common words) and for this we will create a list of words separated by .split().**

**Note:- Split function splits a sentences by their whitespaces and returns a list containing words**

In [13]:
review = review.split()
review

['a',
 'wonderful',
 'little',
 'production',
 'the',
 'filming',
 'technique',
 'is',
 'very',
 'unassuming',
 'very',
 'old',
 'time',
 'bbc',
 'fashion',
 'and',
 'gives',
 'a',
 'comforting',
 'and',
 'sometimes',
 'discomforting',
 'sense',
 'of',
 'realism',
 'to',
 'the',
 'entire',
 'piece',
 'the',
 'actors',
 'are',
 'extremely',
 'well',
 'chosen',
 'michael',
 'sheen',
 'not',
 'only',
 'has',
 'got',
 'all',
 'the',
 'polari',
 'but',
 'he',
 'has',
 'all',
 'the',
 'voices',
 'down',
 'pat',
 'too',
 'you',
 'can',
 'truly',
 'see',
 'the',
 'seamless',
 'editing',
 'guided',
 'by',
 'the',
 'references',
 'to',
 'williams',
 'diary',
 'entries',
 'not',
 'only',
 'is',
 'it',
 'well',
 'worth',
 'the',
 'watching',
 'but',
 'it',
 'is',
 'a',
 'terrificly',
 'written',
 'and',
 'performed',
 'piece',
 'a',
 'masterful',
 'production',
 'about',
 'one',
 'of',
 'the',
 'great',
 'master',
 's',
 'of',
 'comedy',
 'and',
 'his',
 'life',
 'the',
 'realism',
 'really',
 'co

In [14]:
import nltk

from nltk.corpus import stopwords

review = [word for word in review if not word in set(stopwords.words('english'))]
review

['wonderful',
 'little',
 'production',
 'filming',
 'technique',
 'unassuming',
 'old',
 'time',
 'bbc',
 'fashion',
 'gives',
 'comforting',
 'sometimes',
 'discomforting',
 'sense',
 'realism',
 'entire',
 'piece',
 'actors',
 'extremely',
 'well',
 'chosen',
 'michael',
 'sheen',
 'got',
 'polari',
 'voices',
 'pat',
 'truly',
 'see',
 'seamless',
 'editing',
 'guided',
 'references',
 'williams',
 'diary',
 'entries',
 'well',
 'worth',
 'watching',
 'terrificly',
 'written',
 'performed',
 'piece',
 'masterful',
 'production',
 'one',
 'great',
 'master',
 'comedy',
 'life',
 'realism',
 'really',
 'comes',
 'home',
 'little',
 'things',
 'fantasy',
 'guard',
 'rather',
 'use',
 'traditional',
 'dream',
 'techniques',
 'remains',
 'solid',
 'disappears',
 'plays',
 'knowledge',
 'senses',
 'particularly',
 'scenes',
 'concerning',
 'orton',
 'halliwell',
 'sets',
 'particularly',
 'flat',
 'halliwell',
 'murals',
 'decorating',
 'every',
 'surface',
 'terribly',
 'well',
 'done']

**Stemming/Lemmatization**
Will apply and observe the differences in both the techniques.

In [15]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

review_stemmer = [stemmer.stem(word) for word in review]
review_stemmer = ' '.join(review_stemmer)
review_stemmer

'wonder littl product film techniqu unassum old time bbc fashion give comfort sometim discomfort sens realism entir piec actor extrem well chosen michael sheen got polari voic pat truli see seamless edit guid refer william diari entri well worth watch terrificli written perform piec master product one great master comedi life realism realli come home littl thing fantasi guard rather use tradit dream techniqu remain solid disappear play knowledg sens particularli scene concern orton halliwel set particularli flat halliwel mural decor everi surfac terribl well done'

In [16]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
review_lemmatize = [lemmatizer.lemmatize(word) for word in review]
review_lemmatize = ' '.join(review_lemmatize)
review_lemmatize

'wonderful little production filming technique unassuming old time bbc fashion give comforting sometimes discomforting sense realism entire piece actor extremely well chosen michael sheen got polari voice pat truly see seamless editing guided reference williams diary entry well worth watching terrificly written performed piece masterful production one great master comedy life realism really come home little thing fantasy guard rather use traditional dream technique remains solid disappears play knowledge sens particularly scene concerning orton halliwell set particularly flat halliwell mural decorating every surface terribly well done'

As seen in both the paragraphs the Stemming does not impart proper meaning to the stem word while in Lemmatizing stem words conveys a meaning ***eg-  fantasi & fantasy respectively.***

**We will use Lemmatized review further**.

Now we will do Vectorization of out text for this we will create a corpus first.

In [17]:
corpus = []
corpus.append(review_lemmatize)
corpus

['wonderful little production filming technique unassuming old time bbc fashion give comforting sometimes discomforting sense realism entire piece actor extremely well chosen michael sheen got polari voice pat truly see seamless editing guided reference williams diary entry well worth watching terrificly written performed piece masterful production one great master comedy life realism really come home little thing fantasy guard rather use traditional dream technique remains solid disappears play knowledge sens particularly scene concerning orton halliwell set particularly flat halliwell mural decorating every surface terribly well done']

**To vectorize we will apply -**
1. Bag of Words model ( CountVectorizer)
2. TF - IDF model (TfidfVectorizer)
3.


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x = cv.fit_transform(corpus).toarray()
x

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1,
        1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1]])

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf  =  TfidfVectorizer()

review_tfidf = tfidf.fit_transform(corpus).toarray()
review_tfidf

array([[0.09712859, 0.09712859, 0.09712859, 0.09712859, 0.09712859,
        0.09712859, 0.09712859, 0.09712859, 0.09712859, 0.09712859,
        0.09712859, 0.09712859, 0.09712859, 0.09712859, 0.09712859,
        0.09712859, 0.09712859, 0.09712859, 0.09712859, 0.09712859,
        0.09712859, 0.09712859, 0.09712859, 0.09712859, 0.09712859,
        0.09712859, 0.09712859, 0.19425717, 0.09712859, 0.09712859,
        0.09712859, 0.19425717, 0.09712859, 0.09712859, 0.09712859,
        0.09712859, 0.09712859, 0.09712859, 0.09712859, 0.19425717,
        0.09712859, 0.09712859, 0.19425717, 0.09712859, 0.09712859,
        0.19425717, 0.09712859, 0.19425717, 0.09712859, 0.09712859,
        0.09712859, 0.09712859, 0.09712859, 0.09712859, 0.09712859,
        0.09712859, 0.09712859, 0.09712859, 0.09712859, 0.09712859,
        0.09712859, 0.19425717, 0.09712859, 0.09712859, 0.09712859,
        0.09712859, 0.09712859, 0.09712859, 0.09712859, 0.09712859,
        0.09712859, 0.09712859, 0.29138576, 0.09

**Now we will apply the techiques on whole dataset keeping aside 25% of data for testing purposes**

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset['review'], dataset['sentiment'], test_size=0.25, random_state=42)


**Converting sentiments into numerical form**

In [44]:
y_train = y_train.replace({'positive':1, 'negative':0})
y_test = y_test.replace({'positive':1, 'negative':0})

**Cleaning text and forming train and test corpus**

In [22]:
x_train.iloc[1]

"This is the kind of movie that wants to be good but sucks. First thing, what the hell are those punk trying to do with the school? I think the kids doesn't seem to realize the gravity of the situation. Deker guy say to the girl that they under his responsibility when she ask why he wants to go back for them but right after this he gives a gun to the wheel chair dude and wants him to go alone repair the phone line. Where is the responsibility there? I understand poor actors must pay their food but why not just give them the money that takes to make a stupid movie like that or give that money to a charity. Oh yea and none of them knows how to aim. The stupid punk guy shoots in the cafeteria nowhere like a crazy. They all want to look professional but they all suck. One more thing I don't believe that there's no emergency exit in the school the kids are trying several doors but they all locked. What happens if there's a fire and the dumass security guard is dead? It is illegal to not hav

In [23]:
import re

In [24]:
corpus_train = []
corpus_test = []

for i in range(x_train.shape[0]):
    soup = BeautifulSoup(x_train.iloc[i],'html.parser')
    review = soup.get_text()
    review = re.sub('/[[^]]*/]',' ',review)
    review = re.sub('[^a-zA-Z]',' ',review)
    review = review.lower()
    review = review.split()
    
    lm  = WordNetLemmatizer()
    review = [lm.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus_train.append(review)


for j in range(x_test.shape[0]):
    soup = BeautifulSoup(x_test.iloc[j],'html.parser')
    review = soup.get_text()
    review = re.sub('/[[^]]*/]',' ',review)
    review = re.sub('[^a-zA-Z]',' ',review)
    review = review.lower()
    review = review.split()
    
    lm = WordNetLemmatizer()
    review = [lm.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus_test.append(review)
    
    
    


    
    
  

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Possible nested set at position 2
  import sys


**Validating sample entry**

In [25]:
corpus_train[-1]

'decent movie although little bit short time pack lot action grit commonsense emotion time frame matt dillon main character great job movie emotion intensity convincing tense throughout movie typical fancy expensive hollywood cgi action movie satisfying movie indeed price evening great movie movie straight traditional action movie great acting story directing would recommend movie character development character good make believe actually seeing real event taking place movie believe made cheaper budget acting quality much higher'

In [26]:
corpus_test[-1]

'wonderfully funny awe inspiring feature pioneer turntablism dj shadow q bert amazing terrific documentary check every major dj crediting getting scratch thanks herbie hancock post bop classic rockit archival footage complex mind blowing turntable routine time'

**Vectorization using TF-IDF Technique**

In [34]:
tfidf = TfidfVectorizer(ngram_range = (1, 3))

tfidf_train = tfidf.fit_transform(corpus_train)
tfidf_test = tfidf.transform(corpus_test)

**Using Linear SupportVectorClassifier(SVC) as first model-**

In [46]:
from sklearn.svm import LinearSVC

linear_svc = LinearSVC(C=0.5, random_state=42)
linear_svc.fit(tfidf_train,y_train)

predict = linear_svc.predict(tfidf_test)


**Performance Metric**
- Classification Report
- Confusion Matrix
- Accuracy Score

In [47]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print('Classification Report: \n', classification_report(y_test, predict, target_names = ['Negative', 'Positive']))
print('Confusion Matrix: \n', confusion_matrix(y_test, predict))
print('Accuracy score: \n', accuracy_score(y_test, predict))

Classification Report: 
               precision    recall  f1-score   support

    Negative       0.91      0.89      0.90      6157
    Positive       0.89      0.92      0.91      6343

    accuracy                           0.90     12500
   macro avg       0.90      0.90      0.90     12500
weighted avg       0.90      0.90      0.90     12500

Confusion Matrix: 
 [[5468  689]
 [ 524 5819]]
Accuracy score: 
 0.90296


In [48]:
predict

array([0, 1, 0, ..., 0, 1, 1])